In [29]:
!pip install psycopg2-binary gspread gspread-dataframe pandas oauth2client

In [30]:
# Celda de Imports (debería ser tu primera o segunda celda de código)
import psycopg2
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
import json
import os
from google.colab import auth
from google.auth import default

In [31]:
import gspread
import json
import os

# Autenticación inteligente: usa la cuenta del robot en GitHub o tu cuenta en Colab
if 'GCP_SA_KEY' in os.environ:
    print("Autenticando con Cuenta de Servicio desde GitHub Secrets...")
    keyfile_dict = json.loads(os.environ['GCP_SA_KEY'])
    gc = gspread.service_account_from_dict(keyfile_dict)
else:
    print("Autenticando con usuario de Colab...")
    from google.colab import auth
    auth.authenticate_user()
    from google.auth import default
    creds, _ = default()
    gc = gspread.authorize(creds)

Autenticando con usuario de Colab...


In [32]:
import os

# Esta sección busca la contraseña.
# Si está en GitHub, la lee de los "secrets".
# Si está en Colab, la lee del gestor de secretos de Colab.
# Si no, la pones a mano (menos seguro).

if 'DB_PASSWORD' in os.environ:
    db_password = os.environ['DB_PASSWORD']
    print("Contraseña leída desde GitHub Secrets.")
else:
    try:
        from google.colab import userdata
        db_password = userdata.get('DB_PASSWORD')
        print("Contraseña leída desde Colab Secrets.")
    except:
        db_password = "TU_CONTRASEÑA_AQUÍ" # <-- Mantén esto para pruebas locales
        print("Usando contraseña local.")

Contraseña leída desde Colab Secrets.


In [33]:
# --------------- CONFIGURACIÓN (Usa la query simple para la prueba) ---------------
db_host = 'ec2-52-206-224-222.compute-1.amazonaws.com'
db_name = 'd77mgnf951v3g0'
db_user = 'reports'
sheet_name = 'Prueba automatización COLAB' # <-- Revisa que el nombre sea exacto
sql_query = """
    SELECT id, name, origin, phone
    FROM stores
    LIMIT 10
"""
# ----------------------------------------------------------------------------------

print("--- INICIANDO PROCESO DE DIAGNÓSTICO ---")
conn = None

try:
    # 1. Conexión a la Base de Datos
    print("Checkpoint 1: Conectando a PostgreSQL...")
    conn = psycopg2.connect(
        host=db_host,
        dbname=db_name,
        user=db_user,
        password=db_password,
        port=5432
    )
    print("Checkpoint 2: Conexión a BBDD exitosa.")

    # 2. Ejecución de la Query
    print("Checkpoint 3: Ejecutando query...")
    df = pd.read_sql_query(sql_query, conn)
    print(f"Checkpoint 4: Query ejecutada. Se encontraron {len(df)} filas.")

    # 3. VERIFICACIÓN CRÍTICA: ¿HAY DATOS?
    if len(df) > 0:
        print("Checkpoint 5: ¡DATOS ENCONTRADOS! Mostrando las primeras 3 filas:")
        print(df.head(3).to_string()) # Imprime una muestra de los datos en el log
    else:
        print("Checkpoint 5: ¡ADVERTENCIA! La consulta no devolvió ninguna fila. El script terminará aquí.")
        # Si no hay datos, no continuamos para no dejar la hoja en blanco.
        raise Exception("La consulta a la base de datos no devolvió resultados.")

    # 4. Conexión con Google Sheets
    print(f"Checkpoint 6: Abriendo Google Sheet: '{sheet_name}'...")
    spreadsheet = gc.open(sheet_name)
    print("Checkpoint 7: Hoja de cálculo abierta.")

    worksheet = spreadsheet.get_worksheet(0)
    print(f"Checkpoint 8: Pestaña '{worksheet.title}' seleccionada.")

    print("Checkpoint 9: Limpiando la hoja y escribiendo datos...")
    worksheet.clear()
    set_with_dataframe(worksheet, df)
    print("Checkpoint 10: Escritura de datos finalizada.")

    print("\n--- ¡PROCESO COMPLETADO CON ÉXITO! ---")

except Exception as e:
    print(f"\n--- ERROR CRÍTICO DETECTADO ---")
    print(f"Detalle del error: {e}")
    raise e

finally:
    if conn is not None:
        conn.close()
        print("Conexión a la base de datos cerrada.")

--- INICIANDO PROCESO DE DIAGNÓSTICO ---
Checkpoint 1: Conectando a PostgreSQL...
Checkpoint 2: Conexión a BBDD exitosa.
Checkpoint 3: Ejecutando query...
Checkpoint 4: Query ejecutada. Se encontraron 10 filas.
Checkpoint 5: ¡DATOS ENCONTRADOS! Mostrando las primeras 3 filas:


/tmp/ipython-input-4044223180.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


       id                          name  origin phone
0  127232   BRAND RISERS CHILE LIMITADA  getnet  None
1   25684        COMERCIAL PRINCESA SPA  getnet  None
2   19765  CLAUDIO JESUS BARRAZA BLANCO  getnet  None
Checkpoint 6: Abriendo Google Sheet: 'Prueba automatización COLAB'...
Checkpoint 7: Hoja de cálculo abierta.
Checkpoint 8: Pestaña 'Hoja 1' seleccionada.
Checkpoint 9: Limpiando la hoja y escribiendo datos...
Checkpoint 10: Escritura de datos finalizada.

--- ¡PROCESO COMPLETADO CON ÉXITO! ---
Conexión a la base de datos cerrada.
